# Gray-Scott model - explicit solution
In following examples we will use the GS model to showcase how it can be represented as Coupled Neural ODEs (CNODEs). But let us first explore the GS model starting with an explicit solution of it. We will be using [SciML](https://sciml.ai/) package [DiffEqFlux.jl`](https://github.com/SciML/DiffEqFlux.jl) and scpecifically [`NeuralODE`](https://docs.sciml.ai/DiffEqFlux/stable/examples/neural_ode/) for defining and solving the problem.

The system that we want to solve, called the the Gray-Scott model, is defined by the following equations:
\begin{equation}\begin{cases} \frac{du}{dt} = D_u \Delta u - uv^2 + f(1-u)  \equiv F_u(u,v) \\ \frac{dv}{dt} = D_v \Delta v + uv^2 - (f+k)v  \equiv G_v(u,v)\end{cases} \end{equation}
where $u(x,y,t):\mathbb{R}^2\times \mathbb{R}\rightarrow \mathbb{R}$ is the concentration of species 1, while $v(x,y,t)$ is the concentration of species 2. This model reproduce the effect of the two species diffusing in their environment and reacting together.
This effect is captured by the ratios between $D_u$ and $D_v$ (the diffusion coefficients) and $f$ and $k$ (the reaction rates).

Let's start creating a grid to discretize the problem. Notice that in literature the coefficients are usually scaled such that $dx=dy=1$, so we will use this scaling to have a direct comparison with literature.

In [ ]:
import CoupledNODE: Grid
dux = duy = dvx = dvy = 1.0
nux = nuy = nvx = nvy = 100
grid_GS_u = Grid(dim = 2, dx = dux, nx = nux, dy = duy, ny = nuy)
grid_GS_v = Grid(dim = 2, dx = dvx, nx = nvx, dy = dvy, ny = nvy)

We start defining a central concentration of $v$ and a constant concentration of $u$:

In [ ]:
function initialize_uv(grid_u, grid_v, u_bkg, v_bkg, center_size)
    u_initial = u_bkg * ones(grid_u.nx, grid_u.ny)
    v_initial = zeros(grid_v.nx, grid_v.ny)
    v_initial[Int(grid_v.nx / 2 - center_size):Int(grid_v.nx / 2 + center_size), Int(grid_v.ny / 2 - center_size):Int(grid_v.ny / 2 + center_size)] .= v_bkg
    return u_initial, v_initial
end
u_initial, v_initial = initialize_uv(grid_GS_u, grid_GS_v, 0.8, 0.9, 4);

We can now define the initial condition as a flattened concatenated array

In [ ]:
uv0 = vcat(reshape(u_initial, grid_GS_u.nx * grid_GS_u.ny, 1),
    reshape(v_initial, grid_GS_v.nx * grid_GS_v.ny, 1));

From the literature, we select the following parameters in order to form nice patterns.

In [ ]:
D_u = 0.16
D_v = 0.08
f = 0.055
k = 0.062;

Define the **right hand sides** of the two equations:

In [ ]:
import CoupledNODE: Laplacian
function F_u(u, v)
    D_u * Laplacian(u, grid_GS_u.dx, grid_GS_u.dy) .- u .* v .^ 2 .+ f .* (1.0 .- u)
end
function G_v(u, v)
    D_v * Laplacian(v, grid_GS_v.dx, grid_GS_v.dy) .+ u .* v .^ 2 .- (f + k) .* v
end

Once the functions have been defined, we can create the CNODE
Notice that in the future, this same constructor will be able to use the user provided neural network to close the equations

In [ ]:
import CoupledNODE: create_f_CNODE
f_CNODE = create_f_CNODE((F_u, G_v), (grid_GS_u, grid_GS_v); is_closed = false);

and we ask Lux for the parameters to train and their structure

In [ ]:
import Lux, Random
rng = Random.seed!(1234);
θ, st = Lux.setup(rng, f_CNODE);

in this example we are not training any parameters, so we can confirm that the vector θ is empty

In [ ]:
length(θ) == 0;

We now do a short *burnout run* to get rid of the initial artifacts. This allows us to discard the transient dynamics and to have a good initial condition for the data collection run.

In [ ]:
using DifferentialEquations: Tsit5
using DiffEqFlux: NeuralODE
trange_burn = (0.0, 10.0);
dt, saveat = (1e-2, 1);
full_CNODE = NeuralODE(f_CNODE,
    trange_burn,
    Tsit5(),
    adaptive = false,
    dt = dt,
    saveat = saveat);
burnout_CNODE_solution = Array(full_CNODE(uv0, θ, st)[1]);

**CNODE run**

We use the output of the *burnout run* to start a longer simulation

In [ ]:
uv0 = burnout_CNODE_solution[:, :, end];
trange = (0.0, 8000.0);

the maximum suggested time step for GS is defined as `1/(4 * Dmax)`

In [ ]:
dt, saveat = (1 / (4 * max(D_u, D_v)), 25);
full_CNODE = NeuralODE(f_CNODE, trange, Tsit5(), adaptive = false, dt = dt, saveat = saveat);
untrained_CNODE_solution = Array(full_CNODE(uv0, θ, st)[1]);

And we unpack the solution to get the two species. Remember that we have concatenated $u$ and $v$ in the same array.

In [ ]:
u = reshape(untrained_CNODE_solution[1:(grid_GS_u.N), :, :],
    grid_GS_u.nx,
    grid_GS_u.ny,
    size(untrained_CNODE_solution, 2),
    :);
v = reshape(untrained_CNODE_solution[(grid_GS_u.N + 1):end, :, :],
    grid_GS_v.nx,
    grid_GS_v.ny,
    size(untrained_CNODE_solution, 2),
    :);

Finally, plot the solution as an animation

In [ ]:
using Plots
anim = Animation()
fig = plot(layout = (1, 2), size = (600, 300))
@gif for i in 1:2:size(u, 4)
    p1 = heatmap(u[:, :, 1, i],
        axis = false,
        bar = true,
        aspect_ratio = 1,
        color = :reds,
        title = "u(x,y)")
    p2 = heatmap(v[:, :, 1, i],
        axis = false,
        bar = true,
        aspect_ratio = 1,
        color = :blues,
        title = "v(x,y)")
    time = round(i * saveat, digits = 0)
    fig = plot(p1, p2, layout = (1, 2), plot_title = "time = $(time)")
    frame(anim, fig)
end
if isdir("./plots")
    gif(anim, "./plots/02.00.GS.gif", fps = 10)
else
    gif(anim, "examples/plots/02.00.GS.gif", fps = 10)
end

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*